In [1]:
import numpy
import numpy as np

import bokeh
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.layouts import column, row
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Spectral11

output_notebook()

Loading BokehJS ...

In [2]:
num_objs = 10

x = 30*np.random.random((num_objs,))
y = 30*np.random.random((num_objs,))

num_pts = np.random.geometric(1.0/8.0, size=num_objs)

xp = []
yp = []
for i in xrange(num_objs):
    xyp_i = np.random.random(num_pts[i])
    xyp_i.sort()
    xyp_i = np.exp(1j * 2*np.pi * xyp_i)
    xyp_i = np.array([np.real(xyp_i), np.imag(xyp_i)])
    xyp_i = xyp_i.transpose(1, 0).copy()

    xp_i = x[i, None] + xyp_i[..., 0]
    yp_i = y[i, None] + xyp_i[..., 1]

    xp.append(xp_i)
    yp.append(yp_i)


t = np.random.random((num_objs,5))
tr = np.repeat([np.arange(t.shape[1])], num_objs, axis=0)
c = Spectral11[:num_objs]

s1 = ColumnDataSource(data=dict(x=xp, y=yp, c=c))
p1 = figure(plot_width=400, plot_height=400, tools=["tap"], title="Select Here")
p1.patches('x', 'y', source=s1, alpha=0.6, color='c')

s2a = ColumnDataSource(data=dict(t=t, tr=tr, c=c))
s2b = ColumnDataSource(data=dict(ts=[], trs=[], cs=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(tr[0][0],tr[0][-1]), y_range=(t.min(),t.max()),
            tools="", title="Watch Here")
p2.multi_line('trs', 'ts', source=s2b, alpha=0.6, color='cs')

s1.callback = CustomJS(args=dict(s2a=s2a, s2b=s2b), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d1 = cb_obj.get('data');
    var d2a = s2a.get('data');
    var d2b = s2b.get('data');

    d2b['trs'] = [];
    d2b['ts'] = [];
    d2b['cs'] = [];

    for (i = 0; i < inds.length; i++) {
        var trs_i = [];
        var ts_i = [];
        for (j = 0; j < d2a['tr'][inds[i]].length; j++) {
            trs_i.push(d2a['tr'][inds[i]][j]);
            ts_i.push(d2a['t'][inds[i]][j]);
        }
        d2b['trs'].push(trs_i);
        d2b['ts'].push(ts_i);
        d2b['cs'].push(d2a['c'][inds[i]]);
    }

    s2b.trigger('change');
""")

layout = row(p1, p2)

show(layout)